In [12]:
import sys

with open('Source/reviews.txt') as r, open('Source/labels.txt') as l:
    raw_reviews = r.readlines()
    raw_labels = l.readlines()
    
 
tokens = list(map(lambda x: set(x.split(" ")), raw_reviews))

vocab_raw = list(set([word for token in tokens for word in token if len(word) > 0 and word[0].isalpha()]))
vocab = list(set([''.join([w for w in word if w.isalpha()]) for word in vocab_raw]))

word2index = {w:i for i, w in enumerate(vocab)}
input_dataset = [list(set([word2index[word] for word in token if word in word2index])) for token in tokens]
    
target_dataset = [int(label == 'positive\n') for label in raw_labels]

In [20]:
import numpy as np
np.random.seed(1)

sigmoid = lambda x: 1 / (1 + np.exp(-x))
sigder = lambda x: x * (1 - x)
alpha, iterations = 0.01, 2
hidden_size = 100

weights_0_1 = 0.2 * np.random.random((len(vocab), hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size, 1)) - 0.1

correct, total = 0, 0
for iter in range(iterations):
    for i in range(len(input_dataset) - 1000):
        x, y = input_dataset[i], target_dataset[i]
        layer_1 = sigmoid(np.sum(weights_0_1[x], axis=0))
        layer_2 = sigmoid(layer_1 @ weights_1_2)

        layer_2_delta = (np.abs(layer_2)- y)
        layer_1_delta = (layer_2_delta @ weights_1_2.T)

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1, layer_2_delta) * alpha

        if np.abs(layer_2 - y) < 0.5:
            correct += 1
        total += 1

        if i % 10 == 9:
            progres = str(i / (len(input_dataset)) * 100)
            sys.stdout.write('\rIter: ' + str(iter) + \
                             ' progress: ' + str(progres)[:5] + \
                             ' accurancy: ' + str(correct / total))

Iter: 1 progress: 95.99 accurancy: 0.86758333333333345

In [35]:
from collections import Counter
import math


def test(inp):
    inp = ''.join([i for i in inp if i.isalpha() or i == ' '])
    x = list(set([word2index[word] for word in inp.split() if word in vocab]))
    layer_1 = sigmoid(np.sum(weights_0_1[x], axis=0))
    layer_2 = sigmoid(layer_1 @ weights_1_2)
    return layer_2


def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)
    

In [31]:
test('awful, not a good one')

array([0.05362158])

In [37]:
similar('fuck')

[('fuck', -0.0),
 ('nooooooooooooooooooooo', -0.61081571500278),
 ('trowing', -0.6209184792392918),
 ('feeney', -0.6216160303227446),
 ('purloin', -0.6342228651177614),
 ('kralik', -0.6493897587262036),
 ('coalescing', -0.652585494710407),
 ('isuh', -0.6536940528939483),
 ('boomtowns', -0.6558293524923691),
 ('brides', -0.6598270454998443)]

In [39]:
import sympy as sp
from sympy.abc import x


f = 1 / (1 - x**3)
f.diff()

3*x**2/(1 - x**3)**2